In [27]:
import pandas as pd
import string
import nltk
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,HashingVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
import re
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.pipeline import make_pipeline
import numpy as np

In [2]:
df1=pd.read_csv('train_text.csv')
df1.head(20)

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
5,f9bf357fe,NaN,NaN,Hal and Chester found ample time to take an in...,-0.861809,0.480936
6,eaf8e7355,NaN,NaN,Hal Paine and Chester Crawford were typical Am...,-1.759061,0.476507
7,0a43a07f1,NaN,NaN,"On the twenty-second of February, 1916, an aut...",-0.952325,0.498116
8,f7eff7419,NaN,NaN,The boys left the capitol and made their way d...,-0.371641,0.463710
9,d96e6dbcd,NaN,NaN,One day he had gone beyond any point which he ...,-1.238432,0.465900


In [3]:
df1['excerpt'].min()

'"\'Morning, Jeeves," I said.\n"Good morning, sir," said Jeeves. He put the good old cup of tea softly on the table by my bed, and I took a refreshing sip. Just right, as usual. Not too hot, not too sweet, not too weak, not too strong, not too much milk, and not a drop spilled in the saucer. A most amazing cove, Jeeves. So dashed competent in every respect. I\'ve said it before, and I\'ll say it again. I mean to say, take just one small instance. Every other valet I\'ve ever had used to barge into my room in the morning while I was still asleep, causing much misery: but Jeeves seems to know when I\'m awake by a sort of telepathy. He always floats in with the cup exactly two minutes after I come to life. Makes a deuce of a lot of difference to a fellow\'s day.\n"How\'s the weather, Jeeves?" \n"Exceptionally clement, sir."\n"Anything in the papers?"\n"Some slight friction threatening in the Balkans, sir. Otherwise, nothing."'

In [4]:
df1.isnull().sum()

id                   0
url_legal         2004
license           2004
excerpt              0
target               0
standard_error       0
dtype: int64

In [10]:
# remove stopwords and special characters
lemma=nltk.WordNetLemmatizer()
df1['length']=df1['excerpt'].str.len()
df1['excerpt']=df1['excerpt'].apply(lambda x: ' '.join([lemma.lemmatize(word) for word in (nltk.word_tokenize(re.sub('[^a-zA-Z0-9]',' ', x.lower()))) if word not in set(stopwords.words('english'))]))
df1['clean_length']=df1['excerpt'].str.len()
df1['prop_removed']=(df1['length']-df1['clean_length'])*100/df1['length']
df1.head()



,id,url_legal,license,excerpt,target,standard_error,length,clean_length,prop_removed
0,c12129c31,NaN,NaN,young people returned ballroom presented decid...,-0.340259,0.464009,591,591,0.0
1,85aa80a4c,NaN,NaN,dinner time mr fayre somewhat silent eye resti...,-0.315372,0.480805,551,551,0.0
2,b69ac6792,NaN,NaN,roger predicted snow departed quickly came two...,-0.580118,0.476676,527,527,0.0
3,dd1000b26,NaN,NaN,outside palace great garden walled round fille...,-1.054013,0.450007,573,573,0.0
4,37c1b32fb,NaN,NaN,upon time three bear lived together house wood...,0.247197,0.510845,413,413,0.0


In [18]:
print(df1['excerpt'].min())

13th september evening columbus first time noticed variation needle phenomenon never remarked first made mention lest people alarmed soon attracted attention pilot filled consternation seemed law nature changing advanced entering another world subject unknown influence apprehended compass lose mysterious virtue without guide become vast trackless ocean columbus tasked science ingenuity reason allay terror told direction needle polar star fixed invisible point variation therefore caused fallacy compass movement north star like heavenly body change revolution every day described circle round pole high opinion entertained columbus profound astronomer gave weight theory alarm subsided


In [11]:
df1['prop_removed'].describe()

count    2834.000000
mean        0.020064
std         0.111747
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         2.352941
Name: prop_removed, dtype: float64

In [12]:
x=df1['excerpt']
y=df1['target']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42)
model=make_pipeline(TfidfVectorizer(binary=False),Ridge())
model.fit(x_train,y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()), ('ridge', Ridge())])

In [15]:
y_pred=model.predict(x_test)


In [17]:
print('MSE : ',mean_squared_error(y_test,y_pred))

MSE :  0.564312655075806


In [ ]:

# training all data and predicting on test cases


In [20]:
df2=pd.read_csv('test_text.csv')
df2['length']=df2['excerpt'].str.len()
df2['excerpt']=df2['excerpt'].apply(lambda x: ' '.join([lemma.lemmatize(word) for word in (nltk.word_tokenize(re.sub('[^a-zA-Z0-9]',' ', x.lower()))) if word not in set(stopwords.words('english'))]))
df2['clean_length']=df2['excerpt'].str.len()
df2['prop_removed']=(df2['length']-df2['clean_length'])*100/df2['length']
df2.head()


,id,url_legal,license,excerpt,length,clean_length,prop_removed
0,c0f722661,NaN,NaN,hope lay jack promise would keep bright light ...,772,478,38.082902
1,f0953f0a5,NaN,NaN,dotty continued go mr gray every night milk so...,967,571,40.951396
2,0df072751,NaN,NaN,bright cheerful scene greeted eye captain raym...,948,550,41.983122
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,cell division process parent cell divide two d...,1144,855,25.262238
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,debugging process finding resolving defect pre...,1094,810,25.959781


In [23]:
model=make_pipeline(TfidfVectorizer(binary=False),Ridge())
model.fit(x,y)
y_pred=model.predict(df2['excerpt'])

In [32]:
pred_data=pd.DataFrame()

pred_data['id']=df2['id']
pred_data['target']=y_pred
pred_data.head()

,id,target
0,c0f722661,-1.420964
1,f0953f0a5,-0.680497
2,0df072751,-0.769024
3,04caf4e0c,-1.482684
4,0e63f8bea,-1.425976


In [26]:
y_pred

array([-1.42096403, -0.68049705, -0.76902374, -1.48268446, -1.42597625,
       -0.85623885,  0.27823713])